# This Notebook Loads Data into MongoDB, a local mySQL Server, and finds one of those tables from the Local Filesystem

1. There will be a separate Azure notebook that accomplishes the data streaming (Bronze, Silver, Gold) and creates the fact table.
2. The dataset was sourced from https://www.kaggle.com/datasets/benten867/cannabis-data?select=consumer+and+producer.csv

In [4]:
import os
from sqlalchemy import text, create_engine
import pandas as pd
import json
import datetime
import pymongo
import requests

In [7]:
kaggle_user = "benten867"
database = "cannabis-data"
table1 = "income.csv"

In [8]:
data_table_url = f"https://www.kaggle.com/{kaggle_user}/{database}/download/{table1}"

In [9]:
response = requests.get(data_table_url)

In [13]:
df = pd.DataFrame(response.raw)

In [14]:
df.head()

""
